In [30]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
import numpy as np
from PIL import Image
import cv2
from sklearn.utils import shuffle

In [33]:
#Load Images from Swing
loadedImages = []
for i in range(0, 900):
    image = cv2.imread('Dataset/SwingImages/swing_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    loadedImages.append(gray_image.reshape(89, 100, 1))

#Load Images From Palm
for i in range(0, 900):
    image = cv2.imread('Dataset/PalmImages/palm_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    loadedImages.append(gray_image.reshape(89, 100, 1))


In [34]:
# Create OutputVector

outputVectors = []
for i in range(0, 900):
    outputVectors.append([1, 0])

for i in range(0, 900):
    outputVectors.append([0, 1])


In [35]:
tf.reset_default_graph()
convnet=input_data(shape=[None,89,100,1],name='input')
convnet=conv_2d(convnet,32,2,activation='relu')
convnet=max_pool_2d(convnet,2)
convnet=conv_2d(convnet,64,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,128,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,256,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,512,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,1024,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,1024,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,128,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,64,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=fully_connected(convnet,1024,activation='relu')
convnet=dropout(convnet,0.75)

convnet=fully_connected(convnet,2,activation='softmax')

convnet=regression(convnet,optimizer='adam',learning_rate=0.001,loss='categorical_crossentropy',name='regression')

model=tflearn.DNN(convnet,tensorboard_verbose=0)


In [37]:
testImages = []
for i in range(900, 1000):
    image = cv2.imread('Dataset/SwingImages/swing_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    testImages.append(gray_image.reshape(89, 100, 1))

#Load Images From Palm
for i in range(900, 1000):
    image = cv2.imread('Dataset/PalmImages/palm_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    testImages.append(gray_image.reshape(89, 100, 1))

testLabels = []

for i in range(900, 1000):
    testLabels.append([1, 0])
for i in range(900, 1000):
    testLabels.append([0, 1])

In [15]:
loadedImages, outputVectors = shuffle(loadedImages, outputVectors, random_state=0)
model.fit(loadedImages, outputVectors, n_epoch=29,
           validation_set = (testImages, testLabels),
           snapshot_step=100, show_metric=True, run_id='convnet_coursera')

model.save("GestureRecogModel.tfl")

Training Step: 825  | total loss: 0.00000 | time: 80.306s
| Adam | epoch: 029 | loss: 0.00000 - acc: 1.0000 -- iter: 1792/1800
Training Step: 826  | total loss: 0.00000 | time: 84.511s
| Adam | epoch: 029 | loss: 0.00000 - acc: 1.0000 | val_loss: 0.00000 - val_acc: 1.0000 -- iter: 1800/1800
--
INFO:tensorflow:/home/sparsha/ProjectWork/gitprojects/Hand Gesture Detection/GestureRecogModel.tfl is not in all_model_checkpoint_paths. Manually adding it.


In [50]:
model.load("GestureRecogModel.tfl")

INFO:tensorflow:Restoring parameters from /home/sparsha/ProjectWork/gitprojects/Hand Gesture Detection/GestureRecogModel.tfl


In [54]:
test = []
for i in range(0, 11):
    image = cv2.imread('Dataset/PalmTest/palm_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    test.append(gray_image.reshape(89, 100, 1))

In [55]:
z = model.predict(test)
print(z)

[[7.4051009e-33 1.0000000e+00]
 [1.3031107e-32 1.0000000e+00]
 [2.4860166e-32 1.0000000e+00]
 [1.3088798e-32 1.0000000e+00]
 [1.1005572e-32 1.0000000e+00]
 [3.7244275e-33 1.0000000e+00]
 [1.3269177e-32 1.0000000e+00]
 [1.3963647e-33 1.0000000e+00]
 [1.3534430e-35 1.0000000e+00]
 [7.9300324e-35 1.0000000e+00]
 [9.2314097e-35 1.0000000e+00]]
